In [1]:
from experta import Fact, KnowledgeEngine, Rule, MATCH
from math import log

output_scores = {
    "BeneishMScore": {},
    "PiotroskiFScore": {},
    "SpringateScore": {},
    "ProfitabilityAnalysis": {}, 
    "SolvencyAnalysis": {},
    "FinancialRatios": {},
    "LiquidityAnalysis":{}
}
# Define the facts for financial variables
class FinancialData(Fact):
    pass

class CombinedScoreEngine(KnowledgeEngine):
    
    # Initialize Piotroski F-Score components
    ni = 0
    roa_score = 0
    cfo = 0
    accrual = 0
    leverage_score = 0
    liquidity = 0
    dilution = 0
    gross_margin = 0
    asset_turnover = 0

    # Beneish M-Score Rules
    @Rule(FinancialData(variable="accounts receivable", current=MATCH.ar_current, previous=MATCH.ar_previous),
          FinancialData(variable="total revenues", current=MATCH.tr_current, previous=MATCH.tr_previous))
    def calculate_dsri(self, ar_current, ar_previous, tr_current, tr_previous):
        dsri = (ar_current / tr_current) / (ar_previous / tr_previous)
        self.declare(FinancialData(variable="DSRI", current=dsri, previous=0))
        output_scores["BeneishMScore"]["DSRI"] = dsri

    @Rule(FinancialData(variable="total revenues", current=MATCH.tr_current, previous=MATCH.tr_previous),
          FinancialData(variable="total costs and expenses", current=MATCH.ce_current, previous=MATCH.ce_previous))
    def calculate_gmi(self, tr_current, tr_previous, ce_current, ce_previous):
        gross_margin_current = (tr_current - ce_current) / tr_current
        gross_margin_previous = (tr_previous - ce_previous) / tr_previous
        gmi = gross_margin_previous / gross_margin_current
        self.declare(FinancialData(variable="GMI", current=gmi, previous=0))
        output_scores["BeneishMScore"]["GMI"] = gmi

    @Rule(FinancialData(variable="total current assets", current=MATCH.tca_current, previous=MATCH.tca_previous),
          FinancialData(variable="net income", current=MATCH.ni_current, previous=MATCH.ni_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_aqi(self, tca_current, tca_previous, ni_current, ni_previous, ta_current, ta_previous):
        aqi = (1 - ((tca_current + ni_current) / ta_current)) / (1 - ((tca_previous + ni_previous) / ta_previous))
        self.declare(FinancialData(variable="AQI", current=aqi, previous=0))
        output_scores["BeneishMScore"]["AQI"] = aqi

    @Rule(FinancialData(variable="total revenues", current=MATCH.tr_current, previous=MATCH.tr_previous))
    def calculate_sgi(self, tr_current, tr_previous):
        sgi = tr_current / tr_previous
        self.declare(FinancialData(variable="SGI", current=sgi, previous=0))
        output_scores["BeneishMScore"]["SGI"] = sgi

    @Rule(FinancialData(variable="debt and finance leases, net of current portion", current=MATCH.dfl_current, previous=MATCH.dfl_previous),
          FinancialData(variable="equity", current=MATCH.e_current, previous=MATCH.e_previous))
    def calculate_depi(self, dfl_current, dfl_previous, e_current, e_previous):
        depreciation_current = dfl_current / (dfl_current + e_current)
        depreciation_previous = dfl_previous / (dfl_previous + e_previous)
        depi = depreciation_previous / depreciation_current
        self.declare(FinancialData(variable="DEPI", current=depi, previous=0))
        output_scores["BeneishMScore"]["DEPI"] = depi

    @Rule(FinancialData(variable="total costs and expenses", current=MATCH.ce_current, previous=MATCH.ce_previous))
    def calculate_sgai(self, ce_current, ce_previous):
        sgai = ce_current / ce_previous
        self.declare(FinancialData(variable="SGAI", current=sgai, previous=0))
        output_scores["BeneishMScore"]["SGAI"] = sgai

    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl_current, previous=MATCH.tl_previous),
          FinancialData(variable="equity", current=MATCH.e_current, previous=MATCH.e_previous))
    def calculate_lvgi(self, tl_current, tl_previous, e_current, e_previous):
        leverage_current = tl_current / (tl_current + e_current)
        leverage_previous = tl_previous / (tl_previous + e_previous)
        lvgi = leverage_current / leverage_previous
        self.declare(FinancialData(variable="LVGI", current=lvgi, previous=0))
        output_scores["BeneishMScore"]["LVGI"] = lvgi

    @Rule(FinancialData(variable="net income", current=MATCH.ni_current),
          FinancialData(variable="cash and cash equivalents", current=MATCH.cash_current),
          FinancialData(variable="total assets", current=MATCH.ta_current))
    def calculate_tata(self, ni_current, cash_current, ta_current):
        tata = (ni_current - cash_current) / ta_current
        self.declare(FinancialData(variable="TATA", current=tata, previous=0))
        output_scores["BeneishMScore"]["TATA"] = tata

    @Rule(FinancialData(variable="DSRI", current=MATCH.dsri),
          FinancialData(variable="GMI", current=MATCH.gmi),
          FinancialData(variable="AQI", current=MATCH.aqi),
          FinancialData(variable="SGI", current=MATCH.sgi),
          FinancialData(variable="DEPI", current=MATCH.depi),
          FinancialData(variable="SGAI", current=MATCH.sgai),
          FinancialData(variable="LVGI", current=MATCH.lvgi),
          FinancialData(variable="TATA", current=MATCH.tata))
    def calculate_beneish_m_score(self, dsri, gmi, aqi, sgi, depi, sgai, lvgi, tata):
        m_score = (-4.84
                   + 0.920 * dsri
                   + 0.528 * gmi
                   + 0.404 * aqi
                   + 0.892 * sgi
                   + 0.115 * depi
                   - 0.172 * sgai
                   + 4.679 * tata
                   - 0.327 * lvgi)
        
        interpretation = "Likely earnings manipulation." if m_score > -2.22 else "Unlikely earnings manipulation."
        
        self.declare(FinancialData(variable="M-Score", current=m_score, previous=0))
        self.declare(FinancialData(variable="Interpretation", current=interpretation, previous=0))
        
        output_scores["BeneishMScore"]["M-Score"] = m_score
        output_scores["BeneishMScore"]["Interpretation"] = interpretation

    # Piotroski F-Score Rules
    @Rule(FinancialData(variable="net income", current=MATCH.ni_current))
    def calculate_ni(self, ni_current):
        self.ni = 1 if ni_current > 0 else 0
        self.declare(FinancialData(variable="NI", current=self.ni, previous=0))
        output_scores["PiotroskiFScore"]["NI"] = self.ni
    
    @Rule(FinancialData(variable="net income", current=MATCH.ni_current),
          FinancialData(variable="total assets", current=MATCH.ta_current))
    def calculate_roa(self, ni_current, ta_current):
        roa = ni_current / ta_current
        self.roa_score = 1 if roa > 0 else 0
        self.declare(FinancialData(variable="ROA", current=self.roa_score, previous=0))
        output_scores["PiotroskiFScore"]["ROA"] = self.roa_score
    
    @Rule(FinancialData(variable="operating profit", current=MATCH.op_current))
    def calculate_cfo(self, op_current):
        self.cfo = 1 if op_current > 0 else 0
        self.declare(FinancialData(variable="CFO", current=self.cfo, previous=0))
        output_scores["PiotroskiFScore"]["CFO"] = self.cfo
    
    @Rule(FinancialData(variable="operating profit", current=MATCH.op_current),
          FinancialData(variable="net income", current=MATCH.ni_current))
    def calculate_accrual(self, op_current, ni_current):
        self.accrual = 1 if op_current > ni_current else 0
        self.declare(FinancialData(variable="Accrual", current=self.accrual, previous=0))
        output_scores["PiotroskiFScore"]["Accrual"] = self.accrual
    
    @Rule(FinancialData(variable="debt and finance leases, net of current portion", current=MATCH.dfl_current, previous=MATCH.dfl_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_leverage(self, dfl_current, dfl_previous, ta_current, ta_previous):
        leverage_current = dfl_current / ta_current
        leverage_previous = dfl_previous / ta_previous
        self.leverage_score = 1 if leverage_current < leverage_previous else 0
        self.declare(FinancialData(variable="Leverage", current=self.leverage_score, previous=0))
        output_scores["PiotroskiFScore"]["Leverage"] = self.leverage_score
    
    @Rule(FinancialData(variable="total current assets", current=MATCH.tca_current, previous=MATCH.tca_previous),
          FinancialData(variable="total current liabilities", current=MATCH.tcl_current, previous=MATCH.tcl_previous))
    def calculate_liquidity(self, tca_current, tca_previous, tcl_current, tcl_previous):
        current_ratio_current = tca_current / tcl_current
        current_ratio_previous = tca_previous / tcl_previous
        self.liquidity = 1 if current_ratio_current > current_ratio_previous else 0
        self.declare(FinancialData(variable="Liquidity", current=self.liquidity, previous=0))
        output_scores["PiotroskiFScore"]["Liquidity"] = self.liquidity
    
    @Rule(FinancialData(variable="total stockholders’ equity", current=MATCH.e_current, previous=MATCH.e_previous))
    def calculate_dilution(self, e_current, e_previous):
        self.dilution = 1 if e_current <= e_previous else 0
        self.declare(FinancialData(variable="Dilution", current=self.dilution, previous=0))
        output_scores["PiotroskiFScore"]["Dilution"] = self.dilution
    
    @Rule(FinancialData(variable="total revenues", current=MATCH.tr_current, previous=MATCH.tr_previous),
          FinancialData(variable="total costs and expenses", current=MATCH.ce_current, previous=MATCH.ce_previous))
    def calculate_gross_margin(self, tr_current, tr_previous, ce_current, ce_previous):
        gross_margin_current = (tr_current - ce_current) / tr_current
        gross_margin_previous = (tr_previous - ce_previous) / tr_previous
        self.gross_margin = 1 if gross_margin_current > gross_margin_previous else 0
        self.declare(FinancialData(variable="Gross Margin", current=self.gross_margin, previous=0))
        output_scores["PiotroskiFScore"]["Gross Margin"] = self.gross_margin
    
    @Rule(FinancialData(variable="total revenues", current=MATCH.tr_current, previous=MATCH.tr_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_asset_turnover(self, tr_current, tr_previous, ta_current, ta_previous):
        asset_turnover_current = tr_current / ta_current
        asset_turnover_previous = tr_previous / ta_previous
        self.asset_turnover = 1 if asset_turnover_current > asset_turnover_previous else 0
        self.declare(FinancialData(variable="Asset Turnover", current=self.asset_turnover, previous=0))
        output_scores["PiotroskiFScore"]["Asset Turnover"] = self.asset_turnover
    
    @Rule(FinancialData(variable="NI", current=MATCH.ni),
          FinancialData(variable="ROA", current=MATCH.roa),
          FinancialData(variable="CFO", current=MATCH.cfo),
          FinancialData(variable="Accrual", current=MATCH.accrual),
          FinancialData(variable="Leverage", current=MATCH.leverage),
          FinancialData(variable="Liquidity", current=MATCH.liquidity),
          FinancialData(variable="Dilution", current=MATCH.dilution),
          FinancialData(variable="Gross Margin", current=MATCH.gross_margin),
          FinancialData(variable="Asset Turnover", current=MATCH.asset_turnover))
    def calculate_f_score(self, ni, roa, cfo, accrual, leverage, liquidity, dilution, gross_margin, asset_turnover):
        f_score = (self.ni + self.roa_score + self.cfo + self.accrual + self.leverage_score +
                   self.liquidity + self.dilution + self.gross_margin + self.asset_turnover)
        output_scores["PiotroskiFScore"]["F-Score"] = f_score
        
        interpretation = self.interpret_f_score(f_score)
        output_scores["PiotroskiFScore"]["Interpretation"] = interpretation
    
    def interpret_f_score(self, f_score):
        if f_score >= 8:
            return "Strong Financial Health"
        elif f_score >= 5:
            return "Average Financial Health"
        else:
            return "Weak Financial Health"

    # Springate Score Rules
    @Rule(FinancialData(variable="total current assets", current=MATCH.tca_current),
          FinancialData(variable="total current liabilities", current=MATCH.tcl_current),
          FinancialData(variable="total assets", current=MATCH.ta_current))
    def calculate_wc_ta(self, tca_current, tcl_current, ta_current):
        working_capital = tca_current - tcl_current
        wc_ta = working_capital / ta_current
        self.declare(FinancialData(variable="WC/TA", current=wc_ta, previous=0))
        output_scores["SpringateScore"]["WC/TA"] = wc_ta
    
    @Rule(FinancialData(variable="operating profit", current=MATCH.op_current),
          FinancialData(variable="total assets", current=MATCH.ta_current))
    def calculate_ebit_ta(self, op_current, ta_current):
        ebit_ta = op_current / ta_current
        self.declare(FinancialData(variable="EBIT/TA", current=ebit_ta, previous=0))
        output_scores["SpringateScore"]["EBIT/TA"] = ebit_ta
    
    @Rule(FinancialData(variable="net income", current=MATCH.ni_current),
          FinancialData(variable="total current liabilities", current=MATCH.tcl_current))
    def calculate_npt_cl(self, ni_current, tcl_current):
        npt_cl = ni_current / tcl_current
        self.declare(FinancialData(variable="NPT/CL", current=npt_cl, previous=0))
        output_scores["SpringateScore"]["NPT/CL"] = npt_cl
    
    @Rule(FinancialData(variable="total revenues", current=MATCH.tr_current),
          FinancialData(variable="total assets", current=MATCH.ta_current))
    def calculate_sales_ta(self, tr_current, ta_current):
        sales_ta = tr_current / ta_current
        self.declare(FinancialData(variable="Sales/TA", current=sales_ta, previous=0))
        output_scores["SpringateScore"]["Sales/TA"] = sales_ta

    @Rule(FinancialData(variable="WC/TA", current=MATCH.wc_ta),
          FinancialData(variable="EBIT/TA", current=MATCH.ebit_ta),
          FinancialData(variable="NPT/CL", current=MATCH.npt_cl),
          FinancialData(variable="Sales/TA", current=MATCH.sales_ta))
    def calculate_springate_score(self, wc_ta, ebit_ta, npt_cl, sales_ta):
        springate_score = (1.03 * wc_ta
                           + 3.07 * ebit_ta
                           + 0.66 * npt_cl
                           + 0.4 * sales_ta)
        output_scores["SpringateScore"]["Springate Score"] = springate_score
        
        interpretation = self.interpret_springate_score(springate_score)
        output_scores["SpringateScore"]["Interpretation"] = interpretation
    
    def interpret_springate_score(self, springate_score):
        if springate_score > 0.862:
            return "The company is in a stable state."
        else:
            return "The company might be under financial stress."

        # New rules for Profitability Analysis
    @Rule(FinancialData(variable="net income", current=MATCH.ni_current, previous=MATCH.ni_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_roa(self, ni_current, ni_previous, ta_current, ta_previous):
        roa_current = ni_current / ta_current
        roa_previous = ni_previous / ta_previous
        self.declare(FinancialData(variable="ROA", current=roa_current, previous=roa_previous))
        output_scores["ProfitabilityAnalysis"]["ROA"] = {
            "Current Year": roa_current,
            "Previous Year": roa_previous
        }

    @Rule(FinancialData(variable="net income", current=MATCH.ni_current, previous=MATCH.ni_previous),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se_current, previous=MATCH.se_previous))
    def calculate_roe(self, ni_current, ni_previous, se_current, se_previous):
        roe_current = ni_current / se_current
        roe_previous = ni_previous / se_previous
        self.declare(FinancialData(variable="ROE", current=roe_current, previous=roe_previous))
        output_scores["ProfitabilityAnalysis"]["ROE"] = {
            "Current Year": roe_current,
            "Previous Year": roe_previous
        }

    @Rule(FinancialData(variable="gross profit", current=MATCH.gp_current, previous=MATCH.gp_previous),
          FinancialData(variable="total revenues", current=MATCH.revenue_current, previous=MATCH.revenue_previous))
    def calculate_gross_margin(self, gp_current, gp_previous, revenue_current, revenue_previous):
        gross_margin_current = gp_current / revenue_current
        gross_margin_previous = gp_previous / revenue_previous
        self.declare(FinancialData(variable="Gross Margin", current=gross_margin_current, previous=gross_margin_previous))
        output_scores["ProfitabilityAnalysis"]["Gross Margin"] = {
            "Current Year": gross_margin_current,
            "Previous Year": gross_margin_previous
        }

    @Rule(FinancialData(variable="operating income", current=MATCH.oi_current, previous=MATCH.oi_previous),
          FinancialData(variable="total revenues", current=MATCH.revenue_current, previous=MATCH.revenue_previous))
    def calculate_operating_profit_margin(self, oi_current, oi_previous, revenue_current, revenue_previous):
        operating_profit_margin_current = oi_current / revenue_current
        operating_profit_margin_previous = oi_previous / revenue_previous
        self.declare(FinancialData(variable="Operating Profit Margin", current=operating_profit_margin_current, previous=operating_profit_margin_previous))
        output_scores["ProfitabilityAnalysis"]["Operating Profit Margin"] = {
            "Current Year": operating_profit_margin_current,
            "Previous Year": operating_profit_margin_previous
        }

    @Rule(FinancialData(variable="net income", current=MATCH.ni_current, previous=MATCH.ni_previous),
          FinancialData(variable="total revenues", current=MATCH.revenue_current, previous=MATCH.revenue_previous))
    def calculate_net_profit_margin(self, ni_current, ni_previous, revenue_current, revenue_previous):
        net_profit_margin_current = ni_current / revenue_current
        net_profit_margin_previous = ni_previous / revenue_previous
        self.declare(FinancialData(variable="Net Profit Margin", current=net_profit_margin_current, previous=net_profit_margin_previous))
        output_scores["ProfitabilityAnalysis"]["Net Profit Margin"] = {
            "Current Year": net_profit_margin_current,
            "Previous Year": net_profit_margin_previous
        }

    @Rule(FinancialData(variable="ROA", current=MATCH.roa_current, previous=MATCH.roa_previous),
          FinancialData(variable="ROE", current=MATCH.roe_current, previous=MATCH.roe_previous),
          FinancialData(variable="Gross Margin", current=MATCH.gross_margin_current, previous=MATCH.gross_margin_previous),
          FinancialData(variable="Operating Profit Margin", current=MATCH.opm_current, previous=MATCH.opm_previous),
          FinancialData(variable="Net Profit Margin", current=MATCH.npm_current, previous=MATCH.npm_previous))
    def evaluate_profitability(self, roa_current, roa_previous, roe_current, roe_previous, gross_margin_current, gross_margin_previous, opm_current, opm_previous, npm_current, npm_previous):
        metrics = {
            "ROA": (roa_current, roa_previous),
            "ROE": (roe_current, roe_previous),
            "Gross Margin": (gross_margin_current, gross_margin_previous),
            "Operating Profit Margin": (opm_current, opm_previous),
            "Net Profit Margin": (npm_current, npm_previous),
        }
        all_ratios_decreased = True
        all_ratios_increased = True

        for metric, (current_value, previous_value) in metrics.items():
            change = current_value - previous_value
            change_percentage = (change / previous_value) * 100 if previous_value != 0 else float('inf')
            change_explanation = self.explain_change1(metric, change, change_percentage)
            
            output_scores["ProfitabilityAnalysis"][metric] = {
                "Current Year": current_value,
                "Previous Year": previous_value,
                "Change": change_percentage,
                "Change Explanation": change_explanation
            }
          

            if change >= 0:
                all_ratios_decreased = False
            if change <= 0:
                all_ratios_increased = False

        if all_ratios_decreased:
            output_scores["ProfitabilityAnalysis"]["Overall Interpretation"] = (
                "Overall, all profitability ratios have decreased, indicating a decline in profitability. "
                "This suggests that the company might be facing challenges in managing costs or generating revenue efficiently."
            )
        elif all_ratios_increased:
            output_scores["ProfitabilityAnalysis"]["Overall Interpretation"] = (
                "Overall, all profitability ratios have increased, indicating an improvement in profitability. "
                "This suggests that the company is effectively managing costs and generating revenue, leading to stronger financial performance."
            )
        else:
            output_scores["ProfitabilityAnalysis"]["Overall Interpretation"] = (
                "Not all profitability ratios have either increased or decreased uniformly. "
                "This suggests a mixed performance in terms of profitability. Further analysis may be needed to understand the areas of strength and weakness."
            )

    @staticmethod
    def explain_change1(metric_name, change, change_percentage):
        if change > 0:
            return f"{metric_name} has increased by {change_percentage:.2f}%, indicating an improvement in the company's performance."
        elif change < 0:
            return f"{metric_name} has decreased by {change_percentage:.2f}%, indicating a decline in the company's performance."
        else:
            return f"{metric_name} has remained unchanged, indicating stability in the company's performance."



  
    # New rules for Solvency Analysis
        # New rules for Solvency Analysis
    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl_current, previous=MATCH.tl_previous),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se_current, previous=MATCH.se_previous))
    def calculate_debt_to_equity(self, tl_current, tl_previous, se_current, se_previous):
        debt_to_equity_current = tl_current / se_current
        debt_to_equity_previous = tl_previous / se_previous
        self.declare(FinancialData(variable="Debt-to-Equity Ratio", current=debt_to_equity_current, previous=debt_to_equity_previous))
        output_scores["SolvencyAnalysis"]["Debt-to-Equity Ratio"] = {
            "Current Year": debt_to_equity_current,
            "Previous Year": debt_to_equity_previous
        }
    
    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl_current, previous=MATCH.tl_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_total_liabilities_to_total_assets(self, tl_current, tl_previous, ta_current, ta_previous):
        liabilities_to_assets_current = tl_current / ta_current
        liabilities_to_assets_previous = tl_previous / ta_previous
        self.declare(FinancialData(variable="Total Liabilities/Total Assets", current=liabilities_to_assets_current, previous=liabilities_to_assets_previous))
        output_scores["SolvencyAnalysis"]["Total Liabilities/Total Assets"] = {
            "Current Year": liabilities_to_assets_current,
            "Previous Year": liabilities_to_assets_previous
        }
    
    @Rule(FinancialData(variable="total stockholders’ equity", current=MATCH.se_current, previous=MATCH.se_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_equity_ratio(self, se_current, se_previous, ta_current, ta_previous):
        equity_ratio_current = se_current / ta_current
        equity_ratio_previous = se_previous / ta_previous
        self.declare(FinancialData(variable="Equity Ratio", current=equity_ratio_current, previous=equity_ratio_previous))
        output_scores["SolvencyAnalysis"]["Equity Ratio"] = {
            "Current Year": equity_ratio_current,
            "Previous Year": equity_ratio_previous
        }
    
    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl_current, previous=MATCH.tl_previous),
          FinancialData(variable="total assets", current=MATCH.ta_current, previous=MATCH.ta_previous))
    def calculate_debt_ratio(self, tl_current, tl_previous, ta_current, ta_previous):
        debt_ratio_current = tl_current / ta_current
        debt_ratio_previous = tl_previous / ta_previous
        self.declare(FinancialData(variable="Debt Ratio", current=debt_ratio_current, previous=debt_ratio_previous))
        output_scores["SolvencyAnalysis"]["Debt Ratio"] = {
            "Current Year": debt_ratio_current,
            "Previous Year": debt_ratio_previous
        }
    
    @Rule(FinancialData(variable="Debt-to-Equity Ratio", current=MATCH.de_current, previous=MATCH.de_previous),
          FinancialData(variable="Total Liabilities/Total Assets", current=MATCH.lta_current, previous=MATCH.lta_previous),
          FinancialData(variable="Equity Ratio", current=MATCH.er_current, previous=MATCH.er_previous),
         )
    def evaluate_solidity(self, de_current, de_previous, lta_current, lta_previous, er_current, er_previous):
        metrics = {
            "Debt-to-Equity Ratio": (de_current, de_previous),
            "Total Liabilities/Total Assets": (lta_current, lta_previous),
            "Equity Ratio": (er_current, er_previous),
#             "Debt Ratio": (dr_current, dr_previous),
        }
        all_ratios_decreased = True
        all_ratios_increased = True

        for metric, (current_value, previous_value) in metrics.items():
            change = current_value - previous_value
            change_percentage = (change / previous_value) * 100 if previous_value != 0 else float('inf')
            change_explanation = self.explain_change1(metric, change, change_percentage)

            output_scores["SolvencyAnalysis"][metric] = {
                "Current Year": current_value,
                "Previous Year": previous_value,
                "Change": change_percentage,
                "Change Explanation": change_explanation
            }


            if change >= 0:
                all_ratios_decreased = False
            if change <= 0:
                all_ratios_increased = False

        if all_ratios_decreased:
            output_scores["SolvencyAnalysis"]["Overall Interpretation"] = (
                "Overall, all solvency ratios have decreased, indicating a decline in the company's ability to meet its long-term debt obligations. "
                "This suggests potential financial distress and may require further investigation into the underlying causes."
            )
        elif all_ratios_increased:
            output_scores["SolvencyAnalysis"]["Overall Interpretation"] = (
                "Overall, all solvency ratios have increased, indicating an improvement in the company's ability to meet its long-term debt obligations. "
                "This suggests a stronger financial position and reduced risk of insolvency."
            )
        else:
            output_scores["SolvencyAnalysis"]["Overall Interpretation"] = (
                "Not all solvency ratios have either increased or decreased uniformly. "
                "This suggests a mixed performance in terms of solvency. Further analysis may be needed to understand the specific areas of strength and weakness."
            )

    @staticmethod
    def explain_change2(metric_name, change, change_percentage):
        if change > 0:
            return f"{metric_name} has increased by {change_percentage:.2f}%, indicating an increase in leverage or financial risk."
        elif change < 0:
            return f"{metric_name} has decreased by {change_percentage:.2f}%, indicating a reduction in leverage or financial risk."
        else:
            return f"{metric_name} has remained unchanged, indicating stability in financial leverage or risk."



    # New rules for Financial Ratios
    @Rule(FinancialData(variable="net income", current=MATCH.ni), 
          FinancialData(variable="total assets", current=MATCH.ta), 
          salience=20)
    def calculate_net_income_to_total_assets(self, ni, ta):
        ratio = ni / ta
        self.declare(FinancialData(variable="net income to total assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Net Income to Total Assets"] = ratio

    @Rule(FinancialData(variable="total stockholders’ equity", current=MATCH.se), 
          FinancialData(variable="total assets", current=MATCH.ta), 
          salience=19)
    def calculate_net_worth_to_assets(self, se, ta):
        ratio = se / ta
        self.declare(FinancialData(variable="net worth to assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Net Worth to Assets"] = ratio

    @Rule(FinancialData(variable="total stockholders’ equity", current=MATCH.se_current, previous=MATCH.se_previous),
          salience=18)
    def calculate_net_value_growth_rate(self, se_current, se_previous):
        growth_rate = ((se_current - se_previous) / se_previous) * 100
        self.declare(FinancialData(variable="net value growth rate", current=growth_rate, previous=0))
        output_scores["FinancialRatios"]["Net Value Growth Rate"] = growth_rate

    @Rule(FinancialData(variable="total current assets", current=MATCH.tca), 
          FinancialData(variable="total current liabilities", current=MATCH.tcl), 
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=17)
    def calculate_working_capital_to_total_assets(self, tca, tcl, ta):
        ratio = (tca - tcl) / ta
        self.declare(FinancialData(variable="working capital to total assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Working Capital to Total Assets"] = ratio

    @Rule(FinancialData(variable="cash and cash equivalents", current=MATCH.cash), 
          FinancialData(variable="total assets", current=MATCH.ta), 
          salience=16)
    def calculate_cash_to_total_assets(self, cash, ta):
        ratio = cash / ta
        self.declare(FinancialData(variable="cash to total assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Cash to Total Assets"] = ratio

    @Rule(FinancialData(variable="cash and cash equivalents", current=MATCH.cash),
          FinancialData(variable="marketable securities", current=MATCH.ms),
          FinancialData(variable="accounts receivable", current=MATCH.ar),
          FinancialData(variable="short-term investments", current=MATCH.sti),
          FinancialData(variable="total current liabilities", current=MATCH.tcl),
          salience=15)
    def calculate_quick_ratio(self, cash, ms, ar, sti, tcl):
        ratio = (cash + ms + ar + sti) / tcl
        self.declare(FinancialData(variable="quick ratio", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Quick Ratio"] = ratio

    @Rule(FinancialData(variable="total debt", current=MATCH.td),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=14)
    def calculate_total_debt_to_net_worth(self, td, se):
        ratio = td / se
        self.declare(FinancialData(variable="total debt to net worth", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Total Debt to Net Worth"] = ratio

    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=13)
    def calculate_debt_ratio(self, tl, ta):
        ratio = tl / ta
        self.declare(FinancialData(variable="debt ratio", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Debt Ratio"] = ratio

    @Rule(FinancialData(variable="total debt", current=MATCH.td),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=12)
    def calculate_borrowing_dependency(self, td, se):
        ratio = td / (td + se)
        self.declare(FinancialData(variable="borrowing dependency", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Borrowing Dependency"] = ratio

    @Rule(FinancialData(variable="cash and cash equivalents", current=MATCH.cash),
          FinancialData(variable="total current liabilities", current=MATCH.tcl),
          salience=11)
    def calculate_cash_to_current_liability(self, cash, tcl):
        ratio = cash / tcl
        self.declare(FinancialData(variable="cash to current liability", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Cash to Current Liability"] = ratio

    @Rule(FinancialData(variable="total current liabilities", current=MATCH.tcl),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=10)
    def calculate_current_liability_to_assets(self, tcl, ta):
        ratio = tcl / ta
        self.declare(FinancialData(variable="current liability to assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Current Liability to Assets"] = ratio

    @Rule(FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          FinancialData(variable="long-term liabilities", current=MATCH.ltl),
          salience=9)
    def calculate_equity_to_long_term_liability(self, se, ltl):
        ratio = se / ltl
        self.declare(FinancialData(variable="equity to long-term liability", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Equity to Long-term Liability"] = ratio

    @Rule(FinancialData(variable="total current liabilities", current=MATCH.tcl),
          FinancialData(variable="total current assets", current=MATCH.tca),
          salience=8)
    def calculate_current_liability_to_current_assets(self, tcl, tca):
        ratio = tcl / tca
        self.declare(FinancialData(variable="current liability to current assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Current Liability to Current Assets"] = ratio

    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=7)
    def calculate_liability_to_equity(self, tl, se):
        ratio = tl / se
        self.declare(FinancialData(variable="liability to equity", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Liability to Equity"] = ratio

    @Rule(FinancialData(variable="total costs and expenses", current=MATCH.tce),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=6)
    def calculate_total_expense_to_assets(self, tce, ta):
        ratio = tce / ta
        self.declare(FinancialData(variable="total expense to assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Total Expense to Assets"] = ratio

    @Rule(FinancialData(variable="inventory", current=MATCH.inv),
          FinancialData(variable="accounts receivable", current=MATCH.ar),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=5)
    def calculate_inventory_and_accounts_receivable_to_net_value(self, inv, ar, se):
        ratio = (inv + ar) / se
        self.declare(FinancialData(variable="inventory and accounts receivable to net value", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Inventory and Accounts Receivable to Net Value"] = ratio

    @Rule(FinancialData(variable="total revenues", current=MATCH.tr),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=4)
    def calculate_net_worth_turnover_rate(self, tr, se):
        ratio = tr / se
        self.declare(FinancialData(variable="net worth turnover rate", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Net Worth Turnover Rate"] = ratio

    @Rule(FinancialData(variable="inventory", current=MATCH.inv),
          FinancialData(variable="total current liabilities", current=MATCH.tcl),
          salience=3)
    def calculate_inventory_to_current_liability(self, inv, tcl):
        ratio = inv / tcl
        self.declare(FinancialData(variable="inventory to current liability", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Inventory to Current Liability"] = ratio

    @Rule(FinancialData(variable="total current liabilities", current=MATCH.tcl),
          FinancialData(variable="total stockholders’ equity", current=MATCH.se),
          salience=2)
    def calculate_current_liabilities_to_equity(self, tcl, se):
        ratio = tcl / se
        self.declare(FinancialData(variable="current liabilities to equity", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Current Liabilities to Equity"] = ratio

    @Rule(FinancialData(variable="long-term liabilities", current=MATCH.ltl),
          FinancialData(variable="total current assets", current=MATCH.tca),
          salience=1)
    def calculate_long_term_liability_to_current_assets(self, ltl, tca):
        ratio = ltl / tca
        self.declare(FinancialData(variable="long-term liability to current assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Long-term Liability to Current Assets"] = ratio

    @Rule(FinancialData(variable="total revenues", current=MATCH.tr),
          FinancialData(variable="total current assets", current=MATCH.tca),
          salience=0)
    def calculate_current_asset_turnover_rate(self, tr, tca):
        ratio = tr / tca
        self.declare(FinancialData(variable="current asset turnover rate", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Current Asset Turnover Rate"] = ratio

    @Rule(FinancialData(variable="total revenues", current=MATCH.tr),
          FinancialData(variable="cash and cash equivalents", current=MATCH.cash),
          FinancialData(variable="short-term investments", current=MATCH.sti),
          FinancialData(variable="accounts receivable, net", current=MATCH.ar),
          salience=-1)
    def calculate_quick_asset_turnover_rate(self, tr, cash, sti, ar):
        quick_assets = cash + sti + ar
        ratio = tr / quick_assets
        self.declare(FinancialData(variable="quick asset turnover rate", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Quick Asset Turnover Rate"] = ratio

    @Rule(FinancialData(variable="retained earnings", current=MATCH.re),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=-2)
    def calculate_retained_earnings_to_total_assets(self, re, ta):
        ratio = re / ta
        self.declare(FinancialData(variable="retained earnings to total assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Retained Earnings to Total Assets"] = ratio

    @Rule(FinancialData(variable="total liabilities", current=MATCH.tl),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=-3)
    def calculate_liability_assets_flag(self, tl, ta):
        flag = 1 if tl > ta else 0
        self.declare(FinancialData(variable="liability-assets flag", current=flag, previous=0))
        output_scores["FinancialRatios"]["Liability-Assets Flag"] = flag

    @Rule(FinancialData(variable="total current liabilities", current=MATCH.tcl),
          FinancialData(variable="total assets", current=MATCH.ta),
          salience=-4)
    def calculate_current_liability_to_assets(self, tcl, ta):
        ratio = tcl / ta
        self.declare(FinancialData(variable="current liability to assets", current=ratio, previous=0))
        output_scores["FinancialRatios"]["Current Liability to Assets"] = ratio
        
        
    @Rule(FinancialData(variable="interest income", current=MATCH.interest_income),
          FinancialData(variable="tax rate", current=MATCH.tax_rate),
          FinancialData(variable="short-term investments", current=MATCH.short_term_investments),
          salience=-5)
    def calculate_after_tax_interest_income(self, interest_income, tax_rate, short_term_investments):

        # Calculate the After-tax Interest Income
        after_tax_income = interest_income * (1 - tax_rate / 100)

        # Calculate the After-tax Interest Income Rate
        after_tax_income_rate = after_tax_income / short_term_investments

        # Calculate the Continuous Interest Rate (after tax) if the after-tax rate is positive
        if after_tax_income_rate >= 0:
            continuous_interest_rate = log(1 + after_tax_income_rate)
        else:
            continuous_interest_rate = None  # Handle this case as appropriate (e.g., set to 0 or leave as None)

        # Declare the result for After-tax Interest Income
        self.declare(FinancialData(variable="after-tax interest income", current=after_tax_income, previous=0))

        # Store the results in output_scores
        output_scores["FinancialRatios"]["After-tax Interest Income Rate"] = after_tax_income_rate
        output_scores["FinancialRatios"]["Continuous Interest Rate (after tax) Rate"] = continuous_interest_rate


    #LiquidityAnalysis
    @Rule(FinancialData(variable="total current assets", current=MATCH.ca_current, previous=MATCH.ca_previous),
          FinancialData(variable="total current liabilities", current=MATCH.cl_current, previous=MATCH.cl_previous))
    def calculate_current_ratio(self, ca_current, ca_previous, cl_current, cl_previous):
        current_ratio_current = ca_current / cl_current
        current_ratio_previous = ca_previous / cl_previous
        output_scores["LiquidityAnalysis"]["Current Ratio"] = {
            "current": current_ratio_current,
            "previous": current_ratio_previous,
            "explanation": f"The current ratio is {current_ratio_current:.2f}:1, meaning that for every unit of current liabilities, the company has {current_ratio_current:.2f} units of current assets to cover it."
        }

    @Rule(FinancialData(variable="total current assets", current=MATCH.ca_current, previous=MATCH.ca_previous),
          FinancialData(variable="inventory", current=MATCH.inventory_current, previous=MATCH.inventory_previous),
          FinancialData(variable="total current liabilities", current=MATCH.cl_current, previous=MATCH.cl_previous))
    def calculate_quick_ratio(self, ca_current, ca_previous, inventory_current, inventory_previous, cl_current, cl_previous):
        quick_ratio_current = (ca_current - inventory_current) / cl_current
        quick_ratio_previous = (ca_previous - inventory_previous) / cl_previous
        output_scores["LiquidityAnalysis"]["Quick Ratio"] = {
            "current": quick_ratio_current,
            "previous": quick_ratio_previous,
            "explanation": f"The quick ratio is {quick_ratio_current:.2f}:1, indicating the company’s ability to cover short-term liabilities using assets that can be quickly converted into cash, excluding inventory."
        }

    @Rule(FinancialData(variable="cash and cash equivalents", current=MATCH.cash_current, previous=MATCH.cash_previous),
          FinancialData(variable="total current liabilities", current=MATCH.cl_current, previous=MATCH.cl_previous))
    def calculate_cash_ratio(self, cash_current, cash_previous, cl_current, cl_previous):
        cash_ratio_current = cash_current / cl_current
        cash_ratio_previous = cash_previous / cl_previous
        output_scores["LiquidityAnalysis"]["Cash Ratio"] = {
            "current": cash_ratio_current,
            "previous": cash_ratio_previous,
            "explanation": f"The cash ratio is {cash_ratio_current:.2f}:1, which directly measures the company’s ability to pay off its short-term liabilities with only cash and cash equivalents."
        }

    # @Rule(FinancialData(variable="total debt", current=MATCH.debt_current, previous=MATCH.debt_previous),
    #       FinancialData(variable="total stockholders’ equity", current=MATCH.equity_current, previous=MATCH.equity_previous))
    # def calculate_debt_to_equity(self, debt_current, debt_previous, equity_current, equity_previous):
    #     debt_to_equity_current = debt_current / equity_current
    #     debt_to_equity_previous = debt_previous / equity_previous
    #     output_scores["LiquidityAnalysis"]["Debt to Equity Ratio"] = {
    #         "current": debt_to_equity_current,
    #         "previous": debt_to_equity_previous,
    #         "explanation": f"The debt to equity ratio is {debt_to_equity_current:.2f}:1, showing the proportion of debt financing compared to shareholders' equity."
    #     }

    @Rule(FinancialData(variable="operating profit", current=MATCH.op_current, previous=MATCH.op_previous),
          FinancialData(variable="interest expense", current=MATCH.ie_current, previous=MATCH.ie_previous))
    def calculate_interest_coverage(self, op_current, op_previous, ie_current, ie_previous):
        interest_coverage_current = op_current / ie_current
        interest_coverage_previous = op_previous / ie_previous
        output_scores["LiquidityAnalysis"]["Interest Coverage Ratio"] = {
            "current": interest_coverage_current,
            "previous": interest_coverage_previous,
            "explanation": f"The interest coverage ratio is {interest_coverage_current:.2f}:1, reflecting the company's ability to cover interest expenses with its operating profits."
        }

    @Rule(FinancialData(variable="total costs and expenses", current=MATCH.cost_current, previous=MATCH.cost_previous),
          FinancialData(variable="inventory", current=MATCH.inventory_current, previous=MATCH.inventory_previous))
    def calculate_inventory_turnover(self, cost_current, cost_previous, inventory_current, inventory_previous):
        inventory_turnover_current = cost_current / inventory_current
        inventory_turnover_previous = cost_previous / inventory_previous
        output_scores["LiquidityAnalysis"]["Inventory Turnover Ratio"] = {
            "current": inventory_turnover_current,
            "previous": inventory_turnover_previous,
            "explanation": f"The inventory turnover ratio is {inventory_turnover_current:.2f}:1, measuring how efficiently inventory is managed."
        }

    @Rule(FinancialData(variable="Current Ratio", current=MATCH.cr_current, previous=MATCH.cr_previous),
          FinancialData(variable="Quick Ratio", current=MATCH.qr_current, previous=MATCH.qr_previous),
          FinancialData(variable="Cash Ratio", current=MATCH.cashr_current, previous=MATCH.cashr_previous))
    def evaluate_liquidity(self, cr_current, cr_previous, qr_current, qr_previous, cashr_current, cashr_previous):
        metrics = {
            "Current Ratio": (cr_current, cr_previous),
            "Quick Ratio": (qr_current, qr_previous),
            "Cash Ratio": (cashr_current, cashr_previous),
        }
        
        all_ratios_decreased = True
        all_ratios_increased = True

        for metric, (current_value, previous_value) in metrics.items():
            change = current_value - previous_value
            change_percentage = (change / previous_value) * 100 if previous_value != 0 else float('inf')
            change_explanation = self.explain_change(metric, change, change_percentage)
            
            output_scores["LiquidityAnalysis"][f"{metric} Change"] = {
                "change_percentage": change_percentage,
                "explanation": change_explanation,
            }

            if change >= 0:
                all_ratios_decreased = False
            if change <= 0:
                all_ratios_increased = False

        if all_ratios_decreased:
            output_scores["LiquidityAnalysis"]["Overall"] = "All liquidity ratios have decreased, indicating a potential decline in the company's liquidity position."
        elif all_ratios_increased:
            output_scores["LiquidityAnalysis"]["Overall"] = "All liquidity ratios have increased, indicating an improvement in the company's liquidity position."
        else:
            output_scores["LiquidityAnalysis"]["Overall"] = "Not all liquidity ratios have either increased or decreased uniformly. This suggests a mixed performance in terms of liquidity."

    @staticmethod
    def explain_change(metric_name, change, change_percentage):
        if change > 0:
            return f"{metric_name} has increased by {change_percentage:.2f}%, indicating improved liquidity."
        elif change < 0:
            return f"{metric_name} has decreased by {change_percentage:.2f}%, indicating weakened liquidity."
        else:
            return f"{metric_name} has remained unchanged, indicating stable liquidity."


def run_combined_scores(financial_dict):
    global output_scores
    output_scores = {
    "BeneishMScore": {},
    "PiotroskiFScore": {},
    "SpringateScore": {},
    "ProfitabilityAnalysis": {}, 
    "SolvencyAnalysis": {}, 
    "FinancialRatios": {},
    "LiquidityAnalysis":{}
}

    # Instantiate the knowledge engine
    engine = CombinedScoreEngine()

    # Reset the engine and declare facts based on financial data
    engine.reset()
    for var, values in financial_dict.items():
        engine.declare(FinancialData(variable=var, current=values[0], previous=values[1]))

    # Run the engine to process the rules and calculate all scores
    engine.run()

    # Return the final dictionary of results
    return output_scores
